## Kodierung Belohnungen

In [1]:
# HIDDEN
import ray
import logging
ray.init(log_to_driver=False, ignore_reinit_error=True, logging_level=logging.ERROR); # logging.FATAL

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

#### Kodierung Belohnungen

- Wir haben jetzt besprochen, wie wichtig es ist, die Beobachtungen zu kodieren.
- Möglicherweise haben wir auch eine gewisse Auswahl im Handlungsraum, obwohl dieser hier (und oft) relativ klar/fest ist.
- Aber was ist mit den Belohnungen? 

#### Aktuelle Einrichtung

- Derzeit erhalten wir eine Belohnung von +1 für das Erreichen des Ziels 
- Das ist ein Teil dessen, was RL so schwer (und beeindruckend) macht:
  - Wir wollen über Aktionen lernen, obwohl wir nicht sofort wissen, ob die Aktion "gut" war 
  - Im Gegensatz dazu steht das überwachte Lernen, bei dem jede Vorhersage, die wir anhand der Trainingsdaten treffen, sofort mit dem bekannten Zielwert verglichen werden kann.


In [2]:
# TODO: perhaps this next slide can be moved to Module 1, since it's very general?

#### Agenten können nicht nur gierig sein

- Können Agenten und Agentinnen einfach lernen, die beste sofortige Belohnung anzustreben?
- Nein. Wenn man dem Nutzer eines Videoempfehlungssystems zum Beispiel ein weiteres lustiges Katzenvideo zeigt, könnte das dazu führen, dass er klickt (hohe unmittelbare Belohnung), aber langfristig das Interesse an dem Dienst verliert (geringe langfristige Belohnung).
- Unser Frozen Lake ist ein weiteres Beispiel für das Problem: Manchmal gibt es überhaupt keine unmittelbare Belohnung, aus der man lernen könnte.

In [3]:
# TODO: perhaps this next section on "Learned action probabilities" could be moved much earlier, even as early as Module 1

#### Gelernte Handlungswahrscheinlichkeiten

- Mit RLlib können wir innerhalb des Modells die Wahrscheinlichkeit jeder Aktion in Abhängigkeit von einer Beobachtung (d.h. der gelernten Policy) betrachten.
- Laden wir das trainierte Modell mit unseren kodierten Beobachtungen:

In [4]:
from envs_03 import RandomLakeObs
from ray.rllib.algorithms.ppo import PPOConfig

ppo_config = (
    PPOConfig()\
    .framework("torch")\
    .rollouts(create_env_on_local_worker=True, horizon=100)\
    .debugging(seed=0, log_level="ERROR")
)
ppo_RandomLakeObs = ppo_config.build(env=RandomLakeObs)

In [5]:
# # HIDDEN

# for i in range(16):
#     ppo_RandomLakeObs.train()
    
# print(ppo_RandomLakeObs.evaluate()["evaluation"]["episode_reward_mean"])

# ppo_RandomLakeObs.save("models/RandomLakeObs-Ray2")

In [6]:
ppo_RandomLakeObs.restore("models/RandomLakeObs-Ray2/checkpoint_000016")

#### Gelernte Handlungswahrscheinlichkeiten

Wir verwenden die Funktion `query_Policy` aus Modul 2:

In [7]:
from utils_03 import query_policy
query_policy(ppo_RandomLakeObs, RandomLakeObs(), [0,0,0,0])

array([0.00902206, 0.5078786 , 0.47434822, 0.00875122], dtype=float32)

- Erinnere dich an die Reihenfolge (links, unten, rechts, oben).
- Wenn die Beobachtung `[0 0 0 0]` ist (keine Löcher oder Kanten in Sicht), geht der Agent lieber nach unten und rechts.

Was ist, wenn es unter dir ein Loch gibt? Wir können eine andere Beobachtung in die Policy einbringen:

In [8]:
query_policy(ppo_RandomLakeObs, RandomLakeObs(), [0,1,0,0])

array([0.01965651, 0.00299437, 0.9645823 , 0.01276694], dtype=float32)

- Jetzt ist es sehr unwahrscheinlich, dass der Agent nach unten geht, und sehr wahrscheinlich, dass er nach rechts geht!
- Auch dies wurde durch Versuch und Irrtum gelernt, und eine Belohnung gab es nur, wenn das Ziel erreicht wurde.

#### Random Lake Belohnungen

- Kann man dem Agenten im Beispiel von Random Lake das Leben nicht leichter machen, indem man ihm sofortige Belohnungen gibt?

Dies ist der aktuelle Belohnungscode:

In [9]:
def reward(self):
    return int(self.player == self.goal)

- Der Agent muss durch Versuch und Irrtum über _ganze Episoden_ lernen, dass es im Allgemeinen gut ist, sich nach unten und rechts zu bewegen 

#### Belohnungen neu definieren

- Lass uns stattdessen versuchen, _bei jedem Schritt eine Belohnung zu geben, die umso höher ist, je näher der Agent dem Ziel kommt_ 

In [10]:
from envs_03 import RandomLakeObs

class RandomLakeObsRew(RandomLakeObs):
    def reward(self):
        return 6-(abs(self.player[0]-self.goal[0]) + abs(self.player[1]-self.goal[1]))

- Bei der obigen Methode wird die [Manhattan-Distanz] (https://en.wikipedia.org/wiki/Taxicab_geometry) zwischen dem Spieler und dem Ziel als Belohnung verwendet 
- Wenn der Agent das Ziel erreicht, erhält er die maximale Belohnung von 6.
- Wenn der Agent am weitesten vom Ziel entfernt ist, erhält er die Mindestbelohnung von 0.

#### Belohnungen neu definieren

In [11]:
env = RandomLakeObsRew()
env.reset()
env.render()
env.reward()

🧑🧊🧊🧊
🧊🧊🧊🧊
🕳🧊🕳🧊
🧊🧊🧊⛳️


0

⬆️ die Belohnung ist 0

⬇️ die Belohnung ist 1, weil wir dem Ziel näher gekommen sind

In [12]:
obs, rew, done, _ = env.step(1)
env.render()
rew

🧊🧊🧊🧊
🧑🧊🧊🧊
🕳🧊🕳🧊
🧊🧊🧊⛳️


1

#### Belohnungen neu definieren

In [13]:
obs, rew, done, _ = env.step(2)
obs, rew, done, _ = env.step(2)
obs, rew, done, _ = env.step(2)
obs, rew, done, _ = env.step(1)
env.render()
rew

🧊🧊🧊🧊
🧊🧊🧊🧊
🕳🧊🕳🧑
🧊🧊🧊⛳️


5

Jetzt ist die Belohnung 5. Als Nächstes wird es 6 sein.

In [14]:
obs, rew, done, _ = env.step(1)
env.render()
rew

🧊🧊🧊🧊
🧊🧊🧊🧊
🕳🧊🕳🧊
🧊🧊🧊🧑


6

#### Belohnungen im Vergleich

- Wir haben also zwei mögliche Belohnungsfunktionen. Welche davon funktioniert besser? 
- Erinnere dich daran, dass wir beim letzten Mal, nachdem wir 8 Iterationen trainiert hatten, das Ziel in etwa 70 % der Zeit erreichen konnten:

In [15]:
ppo_RandomLakeObs.evaluate()['evaluation']['episode_reward_mean']

0.8166666666666667

#### Belohnungen im Vergleich

Lass uns mit der neuen Belohnungsfunktion trainieren!

In [16]:
ppo_RandomLakeObsRew = ppo_config.build(env=RandomLakeObsRew)

In [17]:
for i in range(8):
    ppo_RandomLakeObsRew.train()

In [18]:
ppo_RandomLakeObsRew.evaluate()['evaluation']['episode_reward_mean']

101.90566037735849

Moment mal, was ist hier los??

#### Belohnungen im Vergleich?

- Wir haben versucht, unser RL-System zu verbessern, indem wir die Belohnungsfunktion geformt haben.
- Das wirkte sich (vermutlich) auf das Training aus, aber auch auf unsere Bewertung.
- Beim überwachten Lernen ist das so, als würde man die Bewertungsmetrik vom quadratischen Fehler zum absoluten Fehler ändern.
- Wenn das alte System einen mittleren quadratischen Fehler von 20.000 und das neue System einen mittleren absoluten Fehler von 40 hat, welches System ist dann besser?
- Wir vergleichen hier Äpfel mit Birnen!
- Wir wollen beide Modelle anhand derselben Metrik vergleichen, zum Beispiel der ursprünglichen Metrik 
- Hier wollen wir sehen, wie oft der Agent das Ziel erreicht.

#### Belohnungen im Vergleich?

- Der Code hier ist ein bisschen fortgeschrittener.
- Er ist nur der Vollständigkeit halber enthalten, aber wir werden nicht ins Detail gehen.

In [19]:
from ray.rllib.agents.callbacks import DefaultCallbacks

class MyCallbacks(DefaultCallbacks):
    def on_episode_end(self, *, worker, base_env, policies, episode, env_index, **kwargs):
        info = episode.last_info_for()
        episode.custom_metrics["goal_reached"] = info["player"] == info["goal"]

In [20]:
ppo_config_callback = (
    PPOConfig()\
    .framework("torch")\
    .rollouts(create_env_on_local_worker=True, horizon=100)\
    .debugging(seed=0, log_level="ERROR")\
    .callbacks(callbacks_class=MyCallbacks)\
    .evaluation(evaluation_config={"callbacks" : MyCallbacks})
)

ppo_RandomLakeObsRew = ppo_config_callback.build(env=RandomLakeObsRew)

Der Trainer oben verwendet unser neues Belohnungsschema, meldet/ misst aber auch die Rate der Zielerreichung.

#### Belohnungen im Vergleich?

Lass es uns ausprobieren!

In [21]:
for i in range(8):
    ppo_RandomLakeObsRew.train()

In [22]:
# HIDDEN
ppo_RandomLakeObsRew.evaluate()["evaluation"]["episode_reward_mean"]

101.90566037735849

In [23]:
ppo_RandomLakeObsRew.evaluate()["evaluation"]["custom_metrics"]["goal_reached_mean"]

0.04081632653061224

- Hmm, diese Ergebnisse sind schrecklich!
- Früher hatten wir eine Gewinnquote von 70 % und mehr, und jetzt sind wir fast bei Null.
- Was ist passiert? 🤔

#### Was will der Agent wirklich optimieren?

- Der Agent optimiert in Wirklichkeit die _diskontierte Gesamtbelohnung_.
- _Gesamt_: Er bewertet alle Belohnungen, die er sammelt, nicht nur die letzte Belohnung.
- _Abgezinst_: Er bewertet frühere Belohnungen mehr als spätere.
- Unser Agent maximiert erfolgreich die abgezinste Gesamtbelohnung, aber das entspricht nicht dem Erreichen des Ziels.
- Aber warum? Das Ziel gibt eine höhere Belohnung.

#### Erforschung vs. Ausbeutung

- Ein grundlegendes Konzept im RL ist _Exploration vs. Exploitation_
- Wenn der Agent eine Policy lernt, kann er sich entscheiden, ob er:

1. Dinge tun, von denen er weiß, dass sie ziemlich gut sind ("exploit")
2. Etwas völlig Neues und Verrücktes ausprobieren, nur für den Fall ("explore")

In [24]:
# TODO 
# diagram for this?

#### Erforschung vs. Ausbeutung

- Bei der alten Belohnungsstruktur erhält der Agent eine Belohnung von 0, wenn er das Ziel nicht erreicht.
  - Also versucht er weiter, etwas Besseres zu finden.
- Mit der neuen Belohnungsstruktur erhält der Agent eine Menge Belohnungen, nur weil er herumläuft.
  - Er ist nicht sehr motiviert, die Environment zu erkunden.
- Weil er die abgezinste **Gesamtbelohnung** maximiert, ist es sogar schlecht, das Ziel zu finden!
  - Das führt dazu, dass die Episode endet und die Gesamtbelohnung des Agenten begrenzt wird.
  - Der Agent lernt, das Ziel zu _vermeiden_, vor allem am Anfang der Episode.

#### Eine bessere Belohnungsstruktur entwerfen

- Stattdessen sollten wir versuchen, den Agenten zu bestrafen, wenn er in ein Loch oder über die Kante läuft.
- Es wird einfacher sein, dies direkt in `step` zu implementieren:

In [25]:
class RandomLakeObsRew2(RandomLakeObs):
    def step(self, action):
        # (not shown) existing code gets new_loc, where the player is trying to go
        
        reward = 0
        
        if self.is_valid_loc(new_loc):
            self.player = new_loc
        else:
            reward -= 0.1 # small penalty
            
        if self.holes[self.player]:
            reward -= 0.1 # small penalty
            
        if self.player == self.goal:
            reward += 1
        
        # Return observation/reward/done
        return self.observation(), reward, self.done(), {"player" : self.player, "goal" : self.goal}

In [26]:
# HIDDEN
from envs_03 import RandomLakeObsRew2

#### Nochmal zum Ausprobieren

In [27]:
# HIDDEN
# redefine ppo_RandomLakeObs to include the new callbacks
# so that you can measure the custom metric instead of the reward
# they will give the same value but this is better for consistency
ppo_RandomLakeObs = ppo_config_callback.build(env=RandomLakeObs)

for i in range(8):
    ppo_RandomLakeObs.train()

In [30]:
ppo_RandomLakeObsRew2 = ppo_config_callback.build(env=RandomLakeObsRew2)

In [31]:
for i in range(8):
    ppo_RandomLakeObsRew2.train()

In [32]:
ppo_RandomLakeObs.evaluate()["evaluation"]["custom_metrics"]["goal_reached_mean"]

0.6853448275862069

In [33]:
ppo_RandomLakeObsRew2.evaluate()["evaluation"]["custom_metrics"]["goal_reached_mean"]

0.734982332155477

Es sieht so aus, als ob die beiden Methoden dieses Mal sehr viel ähnlicher abschneiden.

#### Länge der Episode

- Neben der Erfolgsquote können wir auch andere Statistiken über das Verhalten des Agenten berechnen.
- Ein interessantes Maß ist die Episodenlänge.
- RLlib zeichnet diese standardmäßig auf, so dass wir sie leicht abrufen können:

In [34]:
ppo_RandomLakeObs.evaluate()["evaluation"]["episode_len_mean"]

8.585470085470085

In [35]:
ppo_RandomLakeObsRew2.evaluate()["evaluation"]["episode_len_mean"]

7.20216606498195

Obwohl die beiden Mittel die gleiche Erfolgsquote haben, neigt das neue Mittel zu kürzeren Episoden.

Anmerkungen 

- Das ist sehr interessant, weil der Agent den Unterschied zwischen Löchern und Kanten nicht "sehen" kann.
- Wir könnten dies weiter erforschen, indem wir weitere benutzerdefinierte Metriken hinzufügen, z. B. die Anzahl der Beulen in der Kante.

In [ ]:
# TODO
#### disadvantages - loss of generality

#- now only works if goal is at bottom-right
#give a few real-world examples here -> important

## Analogie zum überwachten Lernen: Reward Shaping
<!-- multiple choice -->

Vorhin haben wir eine Analogie zwischen der Kodierung von Beobachtungen im RL und der Vorverarbeitung von Merkmalen beim überwachten Lernen gezogen. Welcher Aspekt des überwachten Lernens ist die beste Analogie zum Reward Shaping im RL?

- [Merkmalstechnik 
- [Modellauswahl | Nicht ganz. Aber wie wir sehen werden, gibt es auch im RL einen Platz für die Modellauswahl!
- [ ] Abstimmung der Hyperparameter | Nicht ganz. Aber wie wir noch sehen werden, gibt es auch im RL einen Platz für Hyperparameter-Tuning!
- [x] Auswahl einer Verlustfunktion | Eine Änderung der Verlustfunktion ändert das "beste" Modell, genau wie eine Änderung der Belohnungen die "beste" Policy ändert.

## Jeden Schritt belohnen: kleine negative Belohnungen
<!-- multiple choice -->

In RL-Environmenten wie Random Lake, wo der Agent ein bestimmtes Ziel erreichen muss, stellen wir uns vor, wir würden für _jeden_ Schritt, den der Agent macht, eine kleine negative Belohnung vergeben. Wie würde sich das auf die Zeit auswirken, die der Agent braucht, bis er das Ziel erreicht?

- [x] Der Agent wird versuchen, das Ziel in so wenigen Schritten wie möglich zu erreichen.
- [Der Agent wird versuchen, das Ziel in so vielen Schritten wie möglich zu erreichen. | Wenn wir jeden Schritt bestrafen, werden mehr Schritte mit einer geringeren Belohnung belohnt.
- [ ] Keine Änderung. | Wenn wir jeden Schritt bestrafen, werden mehr Schritte mit einer geringeren Belohnung belohnt.

## Erkundung vs. Ausbeutung
<!-- multiple choice -->

Welche der folgenden Aussagen über den Kompromiss zwischen Erkundung und Ausbeutung im RL ist richtig?

- [Wenn ein Spieler nur erforscht, wird er nie eine gute Policy finden. | Er wird tatsächlich gute Policyn finden, nur EXTREM langsam.
- [x] Wenn ein Agent nur ausnutzt, wird er nie eine gute Policy finden. | Er kann einfach immer wieder das Gleiche versuchen.
- [Agenten finden immer gute Policyn, auch ohne Exploration/Exploitation.

## Unbeabsichtigte Folgen
<!-- coding exercise -->

In dieser Übung probierst du eine schlechte Idee aus: Jedes Mal, wenn der Agent einen Schritt macht, erhält er eine große negative Belohnung. Wir werden -1 pro Schritt verwenden. Für das Erreichen des Ziels erhält der Agent immer noch eine Belohnung von +1. Implementiere diese Belohnung, trainiere den Agenten und schau dir die durchschnittliche Episodenlänge an, die der Code ausgibt. Vergleiche dies mit der durchschnittlichen Episodenlänge eines Agenten, der nur nach dem Zufallsprinzip handelt. Beantworte dann die Multiple-Choice-Frage zum Verhalten des Agenten. Was denkst du, was hier vor sich geht? 

(Zu deiner Information: Wie bereits erwähnt, kann diese Art der Veränderung einer Environment auch mit Gym-Wrappern erreicht werden)

In [ ]:
# EXERCISE
from utils_03 import lake_default_config
from envs_03 import RandomLakeObs

class RandomLakeBadIdea(RandomLakeObs):
    def reward(self):
        old_reward = int(self.player == self.goal) 
        return ____
    
ppo = lake_default_config.build(env=____)

for i in range(8):
    ppo.train()
    
print("Average episode length for trained agent: %.1f" % 
      ppo.evaluate()["evaluation"][____])

random_agent_config = (
    lake_default_config\
    .exploration(exploration_config={"type": "Random"})\
    .evaluation(evaluation_config={"explore" : True})
)
random_agent = random_agent_config.build(env=RandomLakeBadIdea)

print("Average episode length for random agent: %.1f" % 
      random_agent.evaluate()["evaluation"][____])

In [2]:
# SOLUTION
from utils_03 import lake_default_config
from envs_03 import RandomLakeObs

class RandomLakeBadIdea(RandomLakeObs):
    def reward(self):
        old_reward = int(self.player == self.goal) 
        return old_reward - 1

ppo = lake_default_config.build(env=RandomLakeBadIdea)


for i in range(8):
    ppo.train()
    
print("Average episode length for trained agent: %.1f" % 
      ppo.evaluate()["evaluation"]["episode_len_mean"])

random_agent_config = (
    lake_default_config\
    .exploration(exploration_config={"type": "Random"})\
    .evaluation(evaluation_config={"explore" : True})
)
random_agent = random_agent_config.build(env=RandomLakeBadIdea)

print("Average episode length for random agent: %.1f" % 
      random_agent.evaluate()["evaluation"]["episode_len_mean"])

0
1
2
3
4
5
6
7
Average episode length for trained agent: 4.4
Average episode length for random agent: 12.1


#### Das Verhalten des Agenten

Was glaubst du, was dieser Agent tut, wenn er in einer Environment trainiert wird, in der es bei jedem Schritt eine große negative Belohnung gibt?

- [ ] Der Agent bleibt still, weil er davon abgehalten wird, sich zu bewegen. | Versuch es noch einmal!
- [ ] Der Agent ist nicht daran interessiert, das Ziel zu erreichen, weil die Belohnung vergleichsweise gering ist. | Versuche es noch einmal!
- [x] Der Agent lernt, so schnell wie möglich in den See zu springen, um die negative Belohnung der Bewegung zu vermeiden. | Pfui! 🥶
- [ ] Der Agent erreicht das Ziel auf Anhieb. | Das wäre aber wünschenswert!